In [1]:
import h5py
import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
%matplotlib inline

In [ ]:
NUM_CLASSES = 8
IMG_SHAPE = (120, 120, 3)
BATCH_SIZE = 8

TRAIN_EPOCH = 100
TRAIN_LR = 1e-3
TRAIN_ES_PATIENCE = 5
TRAIN_LR_PATIENCE = 3
TRAIN_MIN_LR = 1e-6
TRAIN_DROPOUT = 0.1

FT_EPOCH = 500
FT_LR = 1e-5
FT_LR_DECAY_STEP = 80.0
FT_LR_DECAY_RATE = 1
FT_ES_PATIENCE = 20
FT_DROPOUT = 0.2

ES_LR_MIN_DELTA = 0.003
train_acc_list = []
val_acc_list = []
train_loss_list = []
val_loss_list = []
epoch_list = []


In [3]:
# Load your data here, PAtt-Lite was trained with h5py for shorter loading time
with h5py.File('ferp.h5', 'r') as f:
    X_train = f['X_train'][:]  # 形状 (N, 120, 120, 3)
    y_train = f['y_train'][:]  # 形状 (N,)
    X_valid = f['X_val'][:]      # 验证集
    y_valid = f['y_val'][:]      # 验证集标签
    X_test = f['X_test'][:]    # 测试集
    y_test = f['y_test'][:]    # 测试集标签

print("Shape of train_sample: {}".format(X_train.shape))
print("Shape of train_label: {}".format(y_train.shape))
print("Shape of valid_sample: {}".format(X_valid.shape))
print("Shape of valid_label: {}".format(y_valid.shape))
print("Shape of test_sample: {}".format(X_test.shape))
print("Shape of test_label: {}".format(y_test.shape))

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

Shape of train_sample: (66379, 120, 120, 3)
Shape of train_label: (66379,)
Shape of valid_sample: (8341, 120, 120, 3)
Shape of valid_label: (8341,)
Shape of test_sample: (3573, 120, 120, 3)
Shape of test_label: (3573,)


In [4]:
# Model Building
input_layer = tf.keras.Input(shape=IMG_SHAPE, name='universal_input')
sample_resizing = tf.keras.layers.Resizing(224, 224, name="resize")
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip(mode='horizontal'), 
                                        tf.keras.layers.RandomContrast(factor=0.3)], name="augmentation")
preprocess_input = tf.keras.applications.mobilenet.preprocess_input

backbone = tf.keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
backbone.trainable = False
base_model = tf.keras.Model(backbone.input, backbone.layers[-29].output, name='base_model')

self_attention = tf.keras.layers.Attention(use_scale=True, name='attention')
patch_extraction = tf.keras.Sequential([
    tf.keras.layers.SeparableConv2D(256, kernel_size=4, strides=4, padding='same', activation='relu'), 
    tf.keras.layers.SeparableConv2D(256, kernel_size=2, strides=2, padding='valid', activation='relu'), 
    tf.keras.layers.Conv2D(256, kernel_size=1, strides=1, padding='valid', activation='relu')
], name='patch_extraction')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D(name='gap')
pre_classification = tf.keras.Sequential([tf.keras.layers.Dense(32, activation='relu'), 
                                          tf.keras.layers.BatchNormalization()], name='pre_classification')
prediction_layer = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name='classification_head')

inputs = input_layer
x = sample_resizing(inputs)
x = data_augmentation(x)
x = preprocess_input(x)
x = base_model(x, training=False)
x = patch_extraction(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(TRAIN_DROPOUT)(x)
x = pre_classification(x)
#x_exp = tf.keras.layers.Lambda(lambda t: tf.expand_dims(t, axis=1))(x)
#x_attn = self_attention([x_exp, x_exp])
#x = tf.keras.layers.Lambda(lambda t: tf.squeeze(t, axis=1))(x_attn)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs, name='train-head')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=TRAIN_LR, global_clipnorm=3.0), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training Procedure
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=TRAIN_ES_PATIENCE, min_delta=ES_LR_MIN_DELTA, restore_best_weights=True)
learning_rate_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=TRAIN_LR_PATIENCE, verbose=0, min_delta=ES_LR_MIN_DELTA, min_lr=TRAIN_MIN_LR)
history = model.fit(X_train, y_train, epochs=TRAIN_EPOCH, batch_size=BATCH_SIZE, validation_data=(X_valid, y_valid), verbose=1, 
                    class_weight=class_weights, callbacks=[early_stopping_callback, learning_rate_callback])
test_loss, test_acc = model.evaluate(X_test, y_test)

# Model Finetuning
print("\nFinetuning ...")
unfreeze = 59
base_model.trainable = True
fine_tune_from = len(base_model.layers) - unfreeze
for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_from:]:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

inputs = input_layer
x = sample_resizing(inputs)
x = data_augmentation(x)
x = preprocess_input(x)
x = base_model(x, training=False)
x = patch_extraction(x)
x = tf.keras.layers.SpatialDropout2D(FT_DROPOUT)(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(FT_DROPOUT)(x)
x = pre_classification(x)
# 注意这里改为和训练阶段一致的方式处理 attention
#x_exp = tf.keras.layers.Lambda(lambda t: tf.expand_dims(t, axis=1))(x)
#x_attn = self_attention([x_exp, x_exp])
#x = tf.keras.layers.Lambda(lambda t: tf.squeeze(t, axis=1))(x_attn)
x = tf.keras.layers.Dropout(FT_DROPOUT)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs, name='finetune-backbone')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=FT_LR, global_clipnorm=3.0), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training Procedure
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=ES_LR_MIN_DELTA, patience=FT_ES_PATIENCE, restore_best_weights=True)
scheduler = keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=FT_LR, decay_steps=FT_LR_DECAY_STEP, decay_rate=FT_LR_DECAY_RATE)
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(schedule=scheduler)
# 用调度器初始化优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=scheduler)

# 重新编译模型（微调阶段通常会重新编译）
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history_finetune = model.fit(X_train, y_train, epochs=FT_EPOCH, batch_size=BATCH_SIZE, validation_data=(X_valid, y_valid), verbose=1, 
                             initial_epoch=history.epoch[-TRAIN_ES_PATIENCE], callbacks=[early_stopping_callback, tensorboard_callback])
test_loss, test_acc = model.evaluate(X_test, y_test)
model.save('model_FERP.keras')

Epoch 1/100
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 324s 39ms/step - accuracy: 0.5194 - loss: 1.3061 - val_accuracy: 0.6437 - val_loss: 1.1189 - learning_rate: 0.0010
Epoch 2/100
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 386s 47ms/step - accuracy: 0.7290 - loss: 0.7826 - val_accuracy: 0.6144 - val_loss: 1.5102 - learning_rate: 0.0010
Epoch 3/100
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 377s 45ms/step - accuracy: 0.7768 - loss: 0.6479 - val_accuracy: 0.6667 - val_loss: 1.2907 - learning_rate: 0.0010
Epoch 4/100
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 351s 42ms/step - accuracy: 0.8060 - loss: 0.5623 - val_accuracy: 0.6517 - val_loss: 1.4400 - learning_rate: 0.0010
Epoch 5/100
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 328s 40ms/step - accuracy: 0.8264 - loss: 0.5080 - val_accuracy: 0.6693 - val_loss: 1.2613 - learning_rate: 0.0010
Epoch 6/100
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 319s 38ms/step - accuracy: 0.8441 - loss: 0.4492 - val_accuracy: 0.6665 - val_loss: 1.3106 - learning_rate: 0.0010
Epoch 7/100
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 367s 44ms